# Keras Tuner Testing
Keras tuning helps create a better model using all the hyperparameters available. Examples used include https://www.tensorflow.org/tutorials/keras/keras_tuner and https://keras.io/keras_tuner/ 

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pickle
import keras_tuner
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-04-11 16:07:21.330947: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9360] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-11 16:07:21.331063: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-11 16:07:21.331125: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1537] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-11 16:07:21.347299: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Load the Data

In [2]:
#load the data
with open("/share/jcorner_data/pkls/new_mode_svrimg/svrimg_new_mode_train.pkl", "rb") as f:
    (x_train, y_train) = pickle.load(f)

with open("/share/jcorner_data/pkls/new_mode_svrimg/svrimg_new_mode_validation.pkl", "rb") as f:
    (x_val, y_val) = pickle.load(f)

with open("/share/jcorner_data/pkls/new_mode_svrimg/svrimg_new_mode_test.pkl", "rb") as f:
    (x_test, y_test) = pickle.load(f)

num_classes = 4
input_shape = (136, 136, 1)

In [3]:
#Normalize by 80 dBZ
x_train = x_train.astype("float32") / 80
x_test = x_test.astype("float32") / 80
x_val = x_val.astype("float32") / 80

print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_val.shape[0], "validate samples")
print(x_test.shape[0], "test samples")

x_train shape: (1730, 136, 136, 1)
1730 train samples
453 validate samples
299 test samples


In [4]:
#convert to useful information for keras
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

### Keras Testing/Tuning

In [5]:
def build_model(hp):

    #create the model and add input shape
    model = keras.Sequential()
    model.add(keras.Input(shape=input_shape))
    
    #create hyperparameter search for kernel and Conv2D size
    kern_size2d = hp.Int("Kernel Size", min_value=3, max_value=9, step=2)
    conv2d_size = hp.Int("Conv2D Size", min_value=32, max_value=128, step=16)
    model.add(layers.Conv2D(conv2d_size/4, kernel_size=(kern_size2d, kern_size2d), activation="relu"))
    model.add(layers.Conv2D(conv2d_size/2, kernel_size=(kern_size2d, kern_size2d), activation="relu"))
    
    #create hyperparameter search for average pool size
    avg_pool_size = hp.Int("Average Pool Size", min_value=3, max_value=9, step=2)
    model.add(layers.AveragePooling2D(pool_size=(avg_pool_size, avg_pool_size)))
    
    #add the rest of the model
    model.add(layers.Conv2D(conv2d_size, kernel_size=(kern_size2d, kern_size2d), activation="relu"))
    model.add(layers.AveragePooling2D(pool_size=(avg_pool_size, avg_pool_size)))
    model.add(layers.Flatten())
    model.add(layers.Dense(num_classes, activation="softmax"))
    
    #create hyperparameter search for learning rate
    #Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning rate', values=[1e-2, 1e-3, 1e-4])
    
    #compile the model
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss="categorical_crossentropy",
                metrics=['accuracy'])

    return model

In [6]:
#create the tuner
tuner = keras_tuner.RandomSearch(build_model,
                     objective='val_accuracy',
                     directory='/share/jcorner_data/tuning',
                     project_name='New_Modes',
                     max_trials=50)

2024-04-11 16:07:24.095569: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-11 16:07:24.102699: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-11 16:07:24.102840: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [7]:
#stop training early if loss stablizes for 5 epochs
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

#rotate/zoom into images to reduce overfitting
datagen = ImageDataGenerator(rotation_range=55, zoom_range=[0.9,1.0], fill_mode="reflect")

In [8]:
#run the hyperparameter search
tuner.search(datagen.flow(x_train, y_train, batch_size=32), epochs=100, validation_data=(x_val, y_val), callbacks=[stop_early])

Trial 50 Complete [00h 00m 14s]
val_accuracy: 0.6490066051483154

Best val_accuracy So Far: 0.9492273926734924
Total elapsed time: 00h 33m 32s


In [9]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal values are:
Last Conv2D Layer .... {best_hps.get('Conv2D Size')} 
Kernel Size .......... {best_hps.get('Kernel Size')}
Learning Rate ........ {best_hps.get('learning rate')}
Pool Size .......... {best_hps.get('Average Pool Size')}
""")


The hyperparameter search is complete. The optimal values are:
Last Conv2D Layer .... 48 
Kernel Size .......... 9
Learning Rate ........ 0.001
Pool Size .......... 3

